In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

In [ ]:
%cd /content/drive/MyDrive/Proactiveware

/content/drive/MyDrive/Proactiveware


In [ ]:
!pwd

/content/drive/MyDrive/Proactiveware


In [ ]:
!pip install geopandas
!pip install earthengine-api
!pip install folium
# !pip install sentinelhub
!pip install gpd
!pip install georaster
!pip install folium Pillow
!pip install geemap
!pip install turfpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import ee
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium import plugins
# import sentinelhub
# import gpd
from geopandas import GeoSeries, GeoDataFrame
import json
import time 
import georaster
import geemap

from turfpy.measurement import bbox
import math
from PIL import Image
import requests
import os



### Authenticate and initialize

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=LJAAxWnVgWLy2ycGLSfXFj14t2WNtr2CIPhhJiY5noQ&tc=IMiJD8nzW21oZNi8QdqUPnmo83yvwBEKtsndScqb0cI&cc=VlRAAmVCkjdGAvPfAL3kiJo3uCzhKnr_P0wN_eSu5Gc

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AbUR2VN7XE4ARQ-7Gy4LwlR7Ch4TZCutjWbvUXJ8Hqd-KyrICvZ1gi5PneA

Successfully saved authorization token.


## Test the API

Test the API by printing the elevation of Mount Everest.

In [ ]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


# Layer for folium

In [ ]:
# Add custom basemaps to folium
basemaps = {
    'Google Maps': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Maps',
        overlay = True,
        control = True
    ),
    'Google Satellite': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = True,
        control = True
    ),
    'Google Satellite Hybrid': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Satellite',
        overlay = True,
        control = True
    ),
    'Esri Satellite': folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = True,
        control = True
    )
}

# Test ploting some Polygon for collect image

In [ ]:
# Opening a GeoJSON file
with open('geojason/selected_polygon.geojson') as f:
    data = json.load(f)

geometry = data['features'][0]['geometry']["geometries"][1]["coordinates"]
polygon = ee.Geometry.Polygon(geometry)


# Define map center and zoom level
centroid = geometry[0][0]
map_center = [centroid[1], centroid[0]]
zoom_level = 100


# Create a folium map object.
my_map = folium.Map(location=map_center, zoom_start=100)

# Add the polygon geometry to the map
folium.GeoJson(
    data=polygon.getInfo(),
    name='Polygon',
    style_function=lambda feature: {
        'fillColor': None,
        'color': 'red',
        'weight': 2
    }
).add_to(my_map)

# Add custom basemaps
basemaps['Google Maps'].add_to(my_map)
basemaps['Google Satellite Hybrid'].add_to(my_map)
display(my_map)

# get image from google map

In [ ]:

path = '/content/drive/MyDrive/Proactiveware/figures'

def num2deg(xtile, ytile, zoom):
    n = 2.0 ** zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

def getXY(lng, lat, zoom):
    tile_size = 256
    numTiles = 1 << zoom
    point_x = (tile_size/ 2 + lng * tile_size / 360.0) * numTiles // tile_size
    sin_y = math.sin(lat * (math.pi / 180.0))
    point_y = ((tile_size / 2) + 0.5 * math.log((1+sin_y)/(1-sin_y)) * -(tile_size / (2 * math.pi))) * numTiles // tile_size
    return int(point_x), int(point_y)


def download_im(geometry, zoom):
    num = 0
    header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}
    minlon, minlat, maxlon, maxlat = bbox(geometry)
    print(minlon, minlat, maxlon, maxlat)

    start_x, start_y = getXY(minlon, maxlat, zoom)
    end_x, end_y = getXY(maxlon, minlat, zoom)


    start_x -=1
    start_y -=1

    end_x +=1
    end_y +=1


    w = (end_x+1 - start_x) * 256
    h = (end_y+1 - start_y) * 256
    result = Image.new("RGB", (w, h))

    for x in range(start_x, end_x+1):
        for y in range(start_y, end_y+1):
            print(f"donwloading {num}/{((end_x+1) - (start_x)) * ((end_y+1) - (start_y)) }")
            
            while True:
                try:
                    url = "http://mt1.google.com/vt/lyrs=y&x={}&y={}&z={}".format(x,y, zoom)
                    raw = requests.get(url, stream=True, headers=header).raw
                    break
                except:
                    continue
            
            i = Image.open(raw)
            x_paste = (x - start_x) * 256
            y_paste = h - (end_y+1 - y) * 256
            result.paste(i, (x_paste, y_paste))
            num +=1
    name = os.path.join(path, "map_{}_{}_{}_{}.png".format(minlat, minlon, maxlat, maxlon))
    result.save(os.path.join (name))
    return result, start_x, start_y, end_x, end_y, w, h


#test to get some polygon
geometry = {'coordinates': [[[-94.57260260212433, 39.14019525724463],
       [-94.57071164476883, 39.14019525724463],
       [-94.57071164476883, 39.141118918340055],
       [-94.57260260212433, 39.141118918340055],
       [-94.57260260212433, 39.14019525724463]]],
     'type': 'Polygon'}
zoom = 22
download_im(geometry, zoom)

-94.57260260212433 39.14019525724463 -94.57071164476883 39.141118918340055
donwloading 0/24
donwloading 1/24
donwloading 2/24
donwloading 3/24
donwloading 4/24
donwloading 5/24
donwloading 6/24
donwloading 7/24
donwloading 8/24
donwloading 9/24
donwloading 10/24
donwloading 11/24
donwloading 12/24
donwloading 13/24
donwloading 14/24
donwloading 15/24
donwloading 16/24
donwloading 17/24
donwloading 18/24
donwloading 19/24
donwloading 20/24
donwloading 21/24
donwloading 22/24
donwloading 23/24


(<PIL.Image.Image image mode=RGB size=1536x1024 at 0x7FBC44AA5F90>,
 124411,
 200106,
 124416,
 200109,
 1536,
 1024)

In [ ]:
# edite multipolygon geojson to polygons
list_files = os.listdir("geojason") 

l = []
list_of_poly = []

for file_ in list_files:
  flag = -1
  print(file_)
  with open(f'geojason/{file_}') as f:
    data = json.load(f)
    l.append(data)

  try:
    x = data["features"][0]["geometry"]["coordinates"][0]
    flag = 1
  except Exception as er:
    flag = 0
    continue

  if flag == 0:
    x = data['features'][0]['geometry']["geometries"][1]["coordinates"]
    list_of_poly.append(x)


  if flag == 1:
    x = data["features"][0]["geometry"]["coordinates"]
    print(len(x))
    for i in range(0,len(x)):
      dic_0 = {
          'coordinates': data["features"][0]["geometry"]["coordinates"][i],
          'type': 'Polygon'
          }
      list_of_poly.append(dic_0)

# print(len(l))

selected_polygon.geojson
geometry_kansas.geojson
24
geometry_st_luis.geojson
62
geometry_limon.geojson
18
4


In [ ]:
for poly in list_of_poly:
  zoom = 19
  download_im(poly, zoom)

-94.5778249455933 39.0906464953322 -94.57667964234395 39.09148961718434
donwloading 0/16
donwloading 1/16
donwloading 2/16
donwloading 3/16
donwloading 4/16
donwloading 5/16
donwloading 6/16
donwloading 7/16
donwloading 8/16
donwloading 9/16
donwloading 10/16
donwloading 11/16
donwloading 12/16
donwloading 13/16
donwloading 14/16
donwloading 15/16
-94.57898634209675 39.09068604942218 -94.57787590756459 39.09150835311103
donwloading 0/16
donwloading 1/16
donwloading 2/16
donwloading 3/16
donwloading 4/16
donwloading 5/16
donwloading 6/16
donwloading 7/16
donwloading 8/16
donwloading 9/16
donwloading 10/16
donwloading 11/16
donwloading 12/16
donwloading 13/16
donwloading 14/16
donwloading 15/16
-94.57887637152714 39.089541053633006 -94.57673328652425 39.09067772224719
donwloading 0/36
donwloading 1/36
donwloading 2/36
donwloading 3/36
donwloading 4/36
donwloading 5/36
donwloading 6/36
donwloading 7/36
donwloading 8/36
donwloading 9/36
donwloading 10/36
donwloading 11/36
donwloading 12/36